In [1]:
# for all execution
!mkdir -p local_logs

In [4]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Access the variables
ACCESS_KEY = os.getenv('ACCESS_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')


In [5]:
!sacctmgr show user $USER withassoc



      User   Def Acct     Admin    Cluster    Account  Partition     Share   Priority MaxJobs MaxNodes  MaxCPUs MaxSubmit     MaxWall  MaxCPUMins                  QOS   Def QOS 
---------- ---------- --------- ---------- ---------- ---------- --------- ---------- ------- -------- -------- --------- ----------- ----------- -------------------- --------- 
      eila   akundaje      None   sherlock   akundaje                  100                                                                               high_p,normal    normal 


In [6]:
# !echo $ACCESS_KEY
# !echo $SECRET_KEY

In [7]:
# check the jobs:

In [8]:
!squeue -u eila


             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [9]:
# !squeue -u eila

In [10]:
# !squeue -u eila

In [11]:
# !scancel -u eila

In [12]:
# !squeue -u eila --name=step1-do


In [13]:
# !chmod +x execute_sbatch_arrays_on_sherlock.py 

# eila_in_download_bams.txt: The input file name.
# step1-download-bams.sh: The SLURM script name.
# 1: entries_per_job (number of entries per job). - how much is each job needs
# 1: max_concurrent_tasks (maximum number of concurrent tasks). - 500 for stanford


# example for input of 1300 lines:
# ./execute_sbatch_arrays_on_sherlock.sh eila_in_download_bams.txt step1-download-bams.sh 500 500
# Breakdown:
# eila_in_download_bams.txt: The input file with 1200 lines.
# step1-download-bams.sh: The SLURM script to run.
# 500: Number of lines per job array (tasks per array).
# 500: Maximum number of concurrent tasks per array.

In [17]:
!ls
# !pwd

01_samtools_merge.sh
01_utils_rsync.sh
02_samtools_sort.sh
03_samtools_index.sh
04_chrombpnet_pipeline.sh
bam_files_list.txt
chrombpnet_tutorial
conda_chrombpnet_installation_cmds.txt
copy_logs_from_local_machine.txt
execute_sbatch_arrays_on_sherlock.sh
extracted_data.csv
intermediate_merged.bam
local_logs
output_file.csv
shared_vars.sh
SLURM_CMDS.ipynb
step1-download-bams.sh
step2-samtool-sort-index.sh
step3-download-peaks.sh
step4-compute-negatives-non-peak-background.sh
step5-1-create_input.sh
step5-2-chrombpnet-pipelline.sh
step6-contrib.ipynb
step6-contrib.sh
steps_inputs


In [16]:
!ls /oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/steps_inputs/step1/eila_encids_with_bam_urls-ENCSR164EBP.txt

/oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/steps_inputs/step1/eila_encids_with_bam_urls-ENCSR164EBP.txt


In [20]:
# step 1
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Access the variables
ACCESS_KEY = os.getenv('ACCESS_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')

# !echo $ACCESS_KEY
# !echo $SECRET_KEY

# !rm ./local_logs/slurm_step1_download_bams*
# !scancel -u eila
# !./execute_sbatch_arrays_on_sherlock.sh /oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/steps_inputs/step1/eila_encids_with_bam_urls.txt step1-download-bams.sh 1000 1000
# !./execute_sbatch_arrays_on_sherlock.sh /oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/steps_inputs/step1/eila_encids_with_bam_urls-debug.txt step1-download-bams.sh 2 2


!./execute_sbatch_arrays_on_sherlock.sh /oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/steps_inputs/step1/eila_encids_with_bam_urls-ENCSR164EBP.txt step1-download-bams.sh 2 2




Total entries in input file: 1
Total number of job arrays to be submitted: 1
Submitting job array from 1 to 1 with a maximum of 2 concurrent tasks.
sbatch: defined options
sbatch: -------------------- --------------------
sbatch: array               : 1-1%2
sbatch: cpus-per-task       : 4
sbatch: error               : /oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/local_logs/slurm_step1_download_bams_err.combined.err
sbatch: mail-type           : INVALID_DEPEND,BEGIN,END,FAIL,REQUEUE,STAGE_OUT
sbatch: mail-user           : eila@stanford.edu
sbatch: mem                 : 4G
sbatch: ntasks              : 1
sbatch: output              : /oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/local_logs/slurm_step1_download_bams_out.combined.out
sbatch: partition           : akundaje,owners
sbatch: time                : 00:35:00
sbatch: verbose             : 1
sbatch: -------------------- --------------------
sbatch: end of defined options
Submitted batch job 53458602


In [86]:
# !ls $GROUP_SCRATCH/$USER/encode_pseudobulks/encode_pseudobulks_data/bams/ENCSR*/*_sorted.bam | head 

In [87]:
# !ls /scratch/users/eila/encode_pseudobulks_data/bam/ENCSR*/*unsorted.bam > /oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/steps_inputs/step2/bam_files_list.txt



In [2]:
# #step 2 - samtools sort and index
# !ls /scratch/users/eila/encode_pseudobulks_data/bam/ENCSR*/*unsorted.bam > /oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/steps_inputs/step2/bam_files_list.txt

# !./execute_sbatch_arrays_on_sherlock.sh /oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/steps_inputs/step2/bam_files_list.txt step2-samtool-sort-index.sh 1000 1000



In [3]:
#step 3 - peaks - independant of BAMS:

# !rm !rm ./local_logs/slurm_step3_peaks*
# !./execute_sbatch_arrays_on_sherlock.sh /oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/steps_inputs/step3/eila_peaks_matched_to_experiments.txt step3-download-peaks.sh 1000 1000

# debug
# !./execute_sbatch_arrays_on_sherlock.sh /oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/steps_inputs/step3/eila_peaks_matched_to_experiments-debug.txt step3-download-peaks.sh 2 2




In [10]:
# !pip install pandas

In [11]:
# # input file for step4:

# import pandas as pd

# # Read the species file
# species_df = pd.read_csv('./steps_inputs/encids_matched_to_species.txt', sep='\t', header=None, names=['ID', 'Species'])

# # Read the data file
# data_df = pd.read_csv('./steps_inputs/step4/peaks_matched_to_experiments.txt', sep=' ', header=None, names=['ID', 'ENCFF', 'Type', 'URL'])

# # Merge the data files based on the ID
# merged_df = pd.merge(data_df, species_df, left_on='ID', right_on='ID', how='left')

# # Save the merged data to a new file
# merged_df.to_csv('./steps_inputs/step4/peaks_matched_to_experiments_species.txt', sep=' ', index=False, header=False)



In [12]:
# #step 4 - non-peaks / background / negative - human:

# debug
# !rm ./local_logs/slurm_step4_negative_*
# !rm -r $SCRATCH/encode_pseudobulks_negative
# !./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/step4/debug_peaks_matched_to_experiments.txt step4-compute-negatives-non-peak-background.sh 2 2

# ## all samples
# ## !./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/step4/peaks_matched_to_experiments_species.txt step4-compute-negatives-non-peak-background.sh 1000 1000


In [277]:

# !rm ./local_logs/slurm.allQuotegpuEp1.01.combined*
# !ls

In [22]:
# prep for step 5
# !chmod +x step5-2-train.sh
# !./step5-1-create_input.sh
# !rm ./local_logs/slurm_step5_2*
# !./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/step5/debug-extracted_paths.txt step5-2-train-chrombpnet-model.sh 2 2




# !./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/step5/debug-one-extracted_paths.txt step5-2-train-chrombpnet-model.sh 1 1


# !./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/step5/extracted_paths.csv step5-2-train-chrombonet-model.sh 1000 1000
# !./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/step5/debug-one-30000-extracted_paths.txt step5-2-train-chrombpnet-model.sh 1 1
# !./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/step5/debug-one-30000-extracted_paths.txt step5-2-train-chrombpnet-model.sh 2 2


File has been generated at: ./steps_inputs/step5/extracted_paths.txt


In [51]:
# Next: 
# array_job_interpret_example.sh - with 1000 peaks. counts and profile - two heads (one after the other)
# seperate the profile from the counts - so we will not have timeput
# note: can also run profile and counts steps separately (will have more jobs, each will take less time)
# This need to be updated. https://github.com/kundajelab/chrombpnet/wiki/Generate-contribution-score-bigwigs
# -pc for counts. -pc for profile. can be as paraemterts with the value of the -pc and one script.
# chrombpnet contribs_bw \
#  -m ${MODEL_PATH} \
#  -r ${REGIONS_PATH} \
#  -g /oak/stanford/groups/akundaje/ziwei75/atac_seq_pipeline/hg38/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta \ # update this for mouse
#  -c /oak/stanford/groups/akundaje/ziwei75/atac_seq_pipeline/hg38/GRCh38_EBV.chrom.sizes.tsv \ # update this for mouse
#  -op ${output_dir}/${ENCSR_ID} | tee -a $log
# comment the counts
# change to $SCRATCH_GOUP
# copy to $SCRATCH_GOUP whatever is remaining.
# 

In [52]:
# !sstat -j 52658719 --format=JobID,MaxRSS,MaxVMSize


In [53]:
# !scontrol show job 52659563


In [1]:
!squeue --user=$USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [81]:
# !scancel -u eila

In [82]:
# !scancel 53227216

In [92]:
!squeue --user=$USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          53430788    owners gpu_ENCS     eila  R    2:02:55      1 sh02-14n15
    53440165_[1%2] owners,ak step3-do     eila PD       0:00      1 (Priority)
53435726_[109-1000 owners,ak step1-do     eila PD       0:00      1 (Resources)
53440137_[1001-135 owners,ak step2-sa     eila PD       0:00      1 (Priority)
53440135_[1-1000%1 owners,ak step2-sa     eila PD       0:00      1 (Priority)


In [84]:
# #step 5 - train chrombpnet - human

# !./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/step5/human/eila_peaks_matched_to_experiments-debug.txt step4-compute-negatives-non-peak-background.sh 2 2




In [21]:
# !ls /scratch/users/eila/encode_pseudobulks/chrombpnet_model

In [22]:
# !ls /scratch/users/eila/encode_pseudobulks/chrombpnet_model/ENCSR396HHA


In [23]:
# !ls /scratch/users/eila/encode_pseudobulks/chrombpnet_model/ENCSR396HHA/negatives/fold_0


In [24]:
# !scontrol show job 52237837
!squeue --user=$USER --state=PENDING | wc -l

3


In [25]:
!squeue --user=$USER --state=COMPLETED | wc -l

1


In [26]:
# !squeue --user=$USER --state=RUNNING

In [196]:
!squeue --user=$USER --state=PENDING


             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          53289142 owners,ak cE-1.01G     eila PD       0:00      1 (Priority)
          53289110 owners,ak cE-1.01G     eila PD       0:00      1 (Priority)
          53278339 owners,ak 04_train     eila PD       0:00      1 (Priority)
          53277920 owners,ak 04_pipel     eila PD       0:00      1 (Priority)


In [28]:
!echo $USER

eila


In [29]:
# !scontrol show job 52275013


In [30]:
# !squeue -u eila


In [31]:
# !squeue --user=$USER --state=PENDING


In [32]:
# !squeue --user=$USER --state=RUNNING


In [33]:
# !scontrol show job | grep $USER
!squeue -u eila --state=FAILED,PREEMPTED,TIMEOUT,PREEMPTED,NODE_FAIL,CANCELLED

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [34]:
# !squeue -u eila -o "%.18i %.9P %.12j %.8u %.2t %.10M %.10l %.10Q %.19V %.19S"


In [35]:
!pwd

/oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas


In [36]:
# !squeue -u eila

In [37]:
# Remove logs
# !find ./local_logs/ -name '*slurm*' -type f -mmin +600 -exec rm {} \;


In [46]:
# !ls /scratch/users/eila/encode_pseudobulks_data/bam/ENCSR925AYP/ENCSR925AYP_sorted.bam 

/scratch/users/eila/encode_pseudobulks_data/bam/ENCSR925AYP/ENCSR925AYP_sorted.bam


In [39]:
# !ls /scratch/users/eila/encode_pseudobulks_data/ENCSR000NVR/peaks/ENCFF585MYS/ENCSR000NVR_ENCFF585MYS.bed.gz


In [47]:
!ls /scratch/users/eila/encode_pseudobulks_data/bam/*/*_sorted.bam | wc -l

1355


In [48]:
!ls /scratch/users/eila/encode_pseudobulks_data/peaks/*/*/*.bed.gz | wc -l


1355


In [49]:
!ls /scratch/users/eila/encode_pseudobulks_data/bam/*/*_unsorted.bam | wc -l

1355


In [50]:
# !rm ./local_logs/slurm_step1_download_bams_err.combined.err

In [51]:
# !rm ./local_logs/slurm_step4*

In [53]:
# !ls

In [54]:
!pwd

/oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas


In [57]:
# !ls -l ./local_logs/slurm_step5*

In [58]:
# bias - given for the train
# train - on one file. provides correlation metrics. we need to look at the motifs that modisco populates
#         model. modisco on subset of the peaks- subset is random. evaludation folder - modisco and interpret on 30K areas
# QC
# interpret on subset of the peaks.


In [59]:
!echo $HOME

/home/users/eila


In [60]:
# %%bash

# # Define the base directories
# SRC_BASE="/scratch/users/eila/encode_pseudobulks_data"
# DEST_BASE="/scratch/users/eila/encode_pseudobulks_data/bam"

# # Loop through all directories matching ENCSR* and move files
# for DIR in ${SRC_BASE}/ENCSR*/bam; do
#     # Extract the directory name after /ENCSR
#     DIR_NAME=$(basename $(dirname "$DIR"))
    
#     # Define source and destination directories
#     SRC_DIR="${SRC_BASE}/${DIR_NAME}/bam"
#     DEST_DIR="${DEST_BASE}/${DIR_NAME}"
    
#     # Create destination directory if it doesn't exist
#     mkdir -p "$DEST_DIR"
    
#     # Move files from source to destination
#     mv "$SRC_DIR"/* "$DEST_DIR"/
    
#     echo "Moved files from $SRC_DIR to $DEST_DIR"
# done


In [15]:
 # scp -r eila@login.sherlock.stanford.edu:scratch/users/eila/encode_pseudobulks_model_training/human/ENCSR037JDN/ENCFF933KCP/fold_1_30000_20240912_182023/evaluation . 


# !rm ./local_logs/slurm_samools_err.combined.err
# !rm ./local_logs/slurm_samools_out.combined.out

rm: cannot remove ‘./local_logs/slurm_samools_err.combined.err’: No such file or directory


In [13]:
# !chmod +x samtools_merge.sh
# !sbatch --verbose samtools_merge.sh
# !ls

sbatch: defined options
sbatch: -------------------- --------------------
sbatch: cpus-per-task       : 16
sbatch: error               : /oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/local_logs/slurm_samools_err.combined.err
sbatch: job-name            : samtools_merge
sbatch: mail-type           : INVALID_DEPEND,BEGIN,END,FAIL,REQUEUE,STAGE_OUT
sbatch: mail-user           : eila@stanford.edu
sbatch: mem                 : 32G
sbatch: ntasks              : 1
sbatch: output              : /oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/local_logs/slurm_samools_out.combined.out
sbatch: partition           : akundaje,owners
sbatch: time                : 01:00:00
sbatch: verbose             : 1
sbatch: -------------------- --------------------
sbatch: end of defined options
Submitted batch job 53184161


In [21]:
!squeue --user=$USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [24]:
# !chmod +x samtools_sort.sh
!sbatch --verbose samtools_sort.sh
# !ls

sbatch: defined options
sbatch: -------------------- --------------------
sbatch: cpus-per-task       : 16
sbatch: error               : /oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/local_logs/slurm_samtools_sort_err.combined.err
sbatch: job-name            : samtools_sort
sbatch: mail-type           : INVALID_DEPEND,BEGIN,END,FAIL,REQUEUE,STAGE_OUT
sbatch: mail-user           : eila@stanford.edu
sbatch: mem                 : 32G
sbatch: ntasks              : 1
sbatch: output              : /oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/local_logs/slurm_samtools_sort_out.combined.out
sbatch: partition           : akundaje,owners
sbatch: time                : 01:00:00
sbatch: verbose             : 1
sbatch: -------------------- --------------------
sbatch: end of defined options
Submitted batch job 53185591


In [37]:
!squeue --user=$USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [ ]:
# while true; do ls -lt; sleep 10; done

In [39]:
# !chmod +x 03_samtools_index.sh
# !sbatch --verbose 03_samtools_index.sh


sbatch: defined options
sbatch: -------------------- --------------------
sbatch: cpus-per-task       : 8
sbatch: error               : /oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/local_logs/slurm_samtools_index_err.combined.err
sbatch: job-name            : samtools_index
sbatch: mail-type           : INVALID_DEPEND,BEGIN,END,FAIL,REQUEUE,STAGE_OUT
sbatch: mail-user           : eila@stanford.edu
sbatch: mem                 : 16G
sbatch: ntasks              : 1
sbatch: output              : /oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas/local_logs/slurm_samtools_index_out.combined.out
sbatch: partition           : akundaje,owners
sbatch: time                : 01:00:00
sbatch: verbose             : 1
sbatch: -------------------- --------------------
sbatch: end of defined options
Submitted batch job 53186108


In [6]:
# !ls
# !chmod +x 04_chrombpnet_pipeline.sh
# !chmod +x 04_chrombpnet_pipeline-inputs.sh
# !chmod +x Step5-2_Debug_chrombpnet_pipeline.sh
# !sbatch --verbose 04_chrombpnet_pipeline.sh

In [7]:
%%bash
GPUS=1

AVAILABLE_GPUS=$(sinfo -p akundaje,owners --format="%G" | grep -o '[0-9]*' | head -n 1)

# Check if GPUs are available on Sherlock
if [ "$AVAILABLE_GPUS" -ge "$GPUS" ]; then
    echo "Submitting with GPU resources"
    # sbatch --gres=gpu:${GPUS} 04_chrombpnet_pipeline.sh
    # sbatch --gres=gpu:${GPUS} 04_chrombpnet_pipeline-inputs.sh
    sbatch --gres=gpu:${GPUS} Step5-2_Debug_chrombpnet_pipeline.sh
else
    echo "No GPU available, submitting with 64 CPUs"
    # sbatch --cpus-per-task=64 04_chrombpnet_pipeline.sh
    # sbatch --cpus-per-task=64 04_chrombpnet_pipeline-inputs.sh
    sbatch --cpus-per-task=64 Step5-2_Debug_chrombpnet_pipeline.sh
fi

Submitting with GPU resources
Submitted batch job 53460996


In [265]:
 # CPU exeuction

# !sbatch --cpus-per-task=64 04_chrombpnet_pipeline.sh
# !sbatch 01_utils_rsync.sh


Submitted batch job 53323168


In [267]:
# !sinfo -p akundaje,owners --format="%G"

In [11]:
!sinfo -p akundaje,owners --format="%G" | grep -o '[0-9]*' | head -n 1

3


In [269]:
# !scontrol update jobid=53289110 nice=0


In [8]:
!squeue -u $USER -o "%.18i %.9u %.2t %.10l %.6p %.20j %.6r"


             JOBID      USER ST TIME_LIMIT PRIORI                 NAME REASON
          53459499      eila  R 2-00:00:00 0.0000      gpu_ENCSR037JDN   None
          53460996      eila PD 2-00:00:00 0.0000             52allval Priori


In [13]:
!squeue -o "%.18i %.9u %.2t %.10l %.6p %.20j %.6r" -u $USER


             JOBID      USER ST TIME_LIMIT PRIORI                 NAME REASON
          53430788      eila PD 2-00:00:00 0.0000      gpu_ENCSR037JDN Priori


In [14]:
!squeue --user=$USER 

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          53430788 owners,ak gpu_ENCS     eila PD       0:00      1 (Priority)


In [15]:
!squeue --user=$USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          53430788 owners,ak gpu_ENCS     eila PD       0:00      1 (Priority)


In [273]:
!squeue --user=$USER --format="%.10i %.10j %.8T %.9m %.10C %.10R %.10N %.10L"

     JOBID       NAME    STATE MIN_MEMOR       CPUS NODELIST(REASON)   NODELIST  TIME_LEFT
  53319606 al2Quotegp  RUNNING       64G          1 sh03-16n09 sh03-16n09 1-21:32:00
  53319609 al2Quotecp  RUNNING       64G         64 sh03-16n09 sh03-16n09 1-21:25:55
  53323168 rsync_bam_  PENDING        8G          1 (Priority)               2:00:00


In [274]:
!squeue --user=$USER --format="%.18i %.50j %.8T %.10M %.5P %.16R"


             JOBID                                               NAME    STATE       TIME PARTI NODELIST(REASON)
          53319606                                  al2QuotegpuEp1.01  RUNNING    2:28:00 akund       sh03-16n09
          53319609                                  al2QuotecpuEp1.01  RUNNING    2:34:05 akund       sh03-16n09
          53323168                                    rsync_bam_files  PENDING       0:00 owner       (Priority)


In [246]:
# !scancel 53305606

In [247]:
!squeue --user=$USER --long



Thu Sep 19 22:44:00 2024
             JOBID PARTITION     NAME     USER    STATE       TIME TIME_LIMI  NODES NODELIST(REASON)
          53305613 owners,ak gpuEp1.0     eila  PENDING       0:00   5:00:00      1 (Priority)
          53305603 owners,ak cpuEp1.0     eila  PENDING       0:00   5:00:00      1 (Priority)


In [248]:
!squeue --user=$USER --format="%.10i %.10j %.8T %.9m %.10C"




     JOBID       NAME    STATE MIN_MEMOR       CPUS
  53305613  gpuEp1.01  PENDING       64G          1
  53305603  cpuEp1.01  PENDING       64G         64


In [10]:
!squeue --user=$USER --format="%.10i %.10j %.8T %.8D %.16R"




     JOBID       NAME    STATE    NODES NODELIST(REASON)
  53459499 gpu_ENCSR0  RUNNING        1       sh03-15n11
  53460996   52allval  RUNNING        1       sh03-17n12


In [57]:
# !ls ./chrombpnet_tutorial/data/splits

In [58]:
# !ls ./chrombpnet_tutorial/data/downloads

In [59]:
# !ls ./chrombpnet_tutorial/bias_model

In [60]:
# !ls ./chrombpnet_tutorial/data

In [47]:
# !rm ./local_logs/slurm_or_*

In [283]:
# !rm ./local_logs/slurm_tr_epoch.combined*

In [25]:
# !rm ./local_logs/slurm.gpu_ENCSR037JDN.*